In [1]:
# ===================== LOAD & CONFIG (Personal OneDrive + Synthetic Data) =====================
from pathlib import Path
import pandas as pd

# --- Paths (Personal OneDrive) ---
CSV_PATH = Path(r"C:\Users\HP\OneDrive\GitHub\Engine_sensor_analytics\synthetic_engine_log.csv")
OUTPUT_DIR = Path(r"C:\Users\HP\OneDrive\GitHub\Engine_sensor_analytics\out")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Load (parse timestamp) ---
df = pd.read_csv(CSV_PATH, parse_dates=["timestamp"])

print("Columns:", df.columns.tolist())
print(df.head())

# ========================== EDA Steps 1–7  =================================
# libraries needed
import os
from typing import Optional
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# =============== USER COLUMNS (for synthetic dataset) ===============
# We’ll use binned RPM as your “speed groups” equivalent of Engine_speed_rpm_binned
if "rpm" in df.columns:
    df["rpm_binned"] = df["rpm"].round(-2).astype("Int64")  # e.g. 1000, 1100, 1200…

ENGINE_SPEED_COL = "rpm_binned"

# Inputs = things you can control/measure upstream
INPUT_COLS = [
    "rpm",
    "torque_Nm",
    "fuel_flow_kgph",
    "air_flow_kgph",
    "lambda",          # NOTE: always use df['lambda'] (keyword)
    "egr_frac",
    "map_bar_abs",
    "mat_C",
    "coolant_in_C",
    "coolant_out_C",
    "oil_T_C",
]

# Outputs = performance/emissions you care about
OUTPUT_COLS = [
    "imep_bar",
    "exhaust_T_C",
    "prr_bar_per_deg",
    "CO2_dry_pct",
    "NOx_ppm",
    "CO_ppm",
    "HC_ppm",
    "knock_index",
]

# Which variables to include for distributions/boxplots
DIST_VARS = OUTPUT_COLS
TOPK = 3  # Top-K inputs per output for scatter

# ----------------- HELPERS  -----------------
def coerce_numeric_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            s = df[c].astype(str)
            s = (s.str.replace('\xa0', '',  regex=False)
                   .str.replace('%',   '',  regex=False)
                   .str.replace(',',   '',  regex=False)
                   .str.replace('\u2212', '-', regex=False)
                   .str.replace('\u2013', '-', regex=False)
                   .str.strip())
            df[c] = pd.to_numeric(s, errors='coerce')
    return df

def load_data(csv_path: Path) -> Optional[pd.DataFrame]:
    if not csv_path.exists():
        print(f"⚠️ CSV not found at {csv_path}. Update CSV_PATH or place your file there.")
        return None
    d = pd.read_csv(csv_path, encoding="utf-8-sig", low_memory=False, parse_dates=["timestamp"])
    # Ensure rpm_binned exists even if we re-load inside main()
    if "rpm" in d.columns and "rpm_binned" not in d.columns:
        d["rpm_binned"] = d["rpm"].round(-2).astype("Int64")
    cols_to_num = sorted(set(INPUT_COLS + OUTPUT_COLS + [ENGINE_SPEED_COL]))
    d = coerce_numeric_cols(d, cols_to_num)
    print(f"Loaded dataset with shape {d.shape}")
    return d

def save_table(df: pd.DataFrame, name: str):
    out = OUTPUT_DIR / f"{name}.csv"
    df.to_csv(out, index=True)
    print(f"Saved table: {out}")

def plot_histogram(series: pd.Series, title: str, fname: str):
    fig, ax = plt.subplots()
    ax.hist(series.dropna().values, bins="auto")
    ax.set_title(title)
    ax.set_xlabel(series.name)
    ax.set_ylabel("Count")
    fig.tight_layout()
    out = OUTPUT_DIR / fname
    fig.savefig(out, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved figure: {out}")

def plot_box_by_speed(df: pd.DataFrame, var: str, speed_col: str, title: str, fname: str):
    if speed_col not in df.columns:
        return
    speeds = sorted(df[speed_col].dropna().unique())
    data = [df.loc[df[speed_col] == s, var].dropna().values for s in speeds]
    fig, ax = plt.subplots()
    ax.boxplot(data, tick_labels=[str(s) for s in speeds], showmeans=True)
    ax.set_title(title)
    ax.set_xlabel(speed_col)
    ax.set_ylabel(var)
    fig.tight_layout()
    out = OUTPUT_DIR / fname
    fig.savefig(out, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved figure: {out}")

def correlation_matrices(df: pd.DataFrame, cols: list[str]) -> tuple[pd.DataFrame, pd.DataFrame]:
    sub = df[cols].select_dtypes(include=[np.number]).copy()
    return sub.corr(method="pearson"), sub.corr(method="spearman")

def corr_with_pvalues(data: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    num = data.select_dtypes(include=[np.number])
    cols = num.columns
    r_mat = pd.DataFrame(np.nan, index=cols, columns=cols, dtype=float)
    p_mat = pd.DataFrame(np.nan, index=cols, columns=cols, dtype=float)
    for i, a in enumerate(cols):
        xa = num[a]
        for j, b in enumerate(cols):
            if j < i:
                r_mat.iloc[i, j] = r_mat.iloc[j, i]
                p_mat.iloc[i, j] = p_mat.iloc[j, i]
                continue
            xb = num[b]
            mask = xa.notna() & xb.notna()
            xv, yv = xa[mask], xb[mask]
            if len(xv) < 3 or xv.nunique() < 2 or yv.nunique() < 2:
                r, p = (np.nan, np.nan)
            else:
                r, p = pearsonr(xv, yv)
            r_mat.iloc[i, j] = r
            p_mat.iloc[i, j] = p
    np.fill_diagonal(r_mat.values, 1.0)
    np.fill_diagonal(p_mat.values, 0.0)
    return r_mat, p_mat

def plot_corr_heatmap(corr: pd.DataFrame, title: str, fname: str):
    fig, ax = plt.subplots()
    ax.imshow(corr.values, aspect="auto")
    ax.set_xticks(range(corr.shape[1])); ax.set_xticklabels(corr.columns, rotation=90)
    ax.set_yticks(range(corr.shape[0])); ax.set_yticklabels(corr.index)
    ax.set_title(title)
    fig.tight_layout()
    out = OUTPUT_DIR / fname
    fig.savefig(out, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved figure: {out}")

def rank_inputs_by_corr(df: pd.DataFrame, inputs: list[str], outputs: list[str]) -> pd.DataFrame:
    rows = []
    for y in outputs:
        if y not in df.columns: continue
        for x in inputs:
            if x not in df.columns: continue
            try:
                r = df[[x, y]].corr(method="pearson").iloc[0, 1]
            except Exception:
                r = np.nan
            rows.append({"output": y, "input": x, "pearson_r": r, "abs_pearson_r": abs(r) if pd.notna(r) else np.nan})
    rank = pd.DataFrame(rows).dropna(subset=["abs_pearson_r"]).sort_values(
        ["output", "abs_pearson_r"], ascending=[True, False]
    )
    return rank

def scatter_xy(df: pd.DataFrame, x: str, y: str, color_by: Optional[str], title: str, fname: str, fit: bool = True) -> None:
    cols = [x, y] + ([color_by] if color_by else [])
    sub_all = df[cols].replace([np.inf, -np.inf], np.nan).dropna()

    fig, ax = plt.subplots()
    if color_by and color_by in sub_all.columns:
        for val, sub in sub_all.groupby(color_by):
            ax.scatter(sub[x], sub[y], s=24, alpha=0.75, label=str(val))
        ax.legend(title=color_by)
    else:
        ax.scatter(sub_all[x], sub_all[y], s=24, alpha=0.75)

    if fit and len(sub_all) >= 2:
        xv, yv = sub_all[x].to_numpy(), sub_all[y].to_numpy()
        m, b = np.polyfit(xv, yv, 1)
        ax.plot([xv.min(), xv.max()], [m * xv.min() + b, m * xv.max() + b], linewidth=2)
        r = np.corrcoef(xv, yv)[0, 1]
        ax.text(0.02, 0.98, f"r = {r:.2f}", transform=ax.transAxes, va="top")

    ax.set_xlabel(x); ax.set_ylabel(y); ax.set_title(title)
    fig.tight_layout()
    out = OUTPUT_DIR / fname
    fig.savefig(out, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved figure: {out}")

def save_per_speed_scatterplots(df, inputs, outputs, speed_col="rpm_binned", outdir=OUTPUT_DIR, rmin=0.60, topk=5):
    if speed_col not in df.columns:
        print(f"{speed_col} not found; skipping per-speed scatter plots.")
        return
    df_num = df.select_dtypes(include=[np.number]).copy()
    bins = sorted(df[speed_col].dropna().unique().tolist())
    def pearson_r(x, y):
        mask = x.notna() & y.notna()
        if mask.sum() < 3: return np.nan
        try: return np.corrcoef(x[mask], y[mask])[0, 1]
        except Exception: return np.nan
    for spd in bins:
        sub = df[df[speed_col] == spd]
        if len(sub) < 3: continue
        sub_num = sub.select_dtypes(include=[np.number])
        for y in outputs:
            if y not in sub_num.columns: continue
            pairs = []
            for x in inputs:
                if x == y or x not in sub_num.columns: continue
                r = pearson_r(sub_num[x], sub_num[y])
                if np.isfinite(r): pairs.append((x, r, abs(r)))
            if not pairs: continue
            pairs.sort(key=lambda t: t[2], reverse=True)
            selected = [p for p in pairs if p[2] >= rmin] or pairs[:topk]
            for x, r, _ in selected:
                m = (sub_num[x].notna() & sub_num[y].notna())
                if m.sum() < 3: continue
                plt.figure(figsize=(6.5, 5))
                plt.scatter(sub_num.loc[m, x], sub_num.loc[m, y], s=22)
                try:
                    mfit, c = np.polyfit(sub_num.loc[m, x], sub_num.loc[m, y], 1)
                    xs = np.linspace(sub_num.loc[m, x].min(), sub_num.loc[m, x].max(), 100)
                    plt.plot(xs, mfit*xs + c)
                except Exception:
                    pass
                plt.title(f"{y} vs {x} (Speed {spd})")
                plt.xlabel(x); plt.ylabel(y)
                plt.text(0.02, 0.95, f"r = {r:.2f}", transform=plt.gca().transAxes, ha="left", va="top")
                out = outdir / f"51_scatter_speed_{spd}_{y}_vs_{x}.png"
                plt.tight_layout(); plt.savefig(out, dpi=150); plt.close()

def pearson_r_series(a, b):
    mask = a.notna() & b.notna()
    if mask.sum() < 3: return np.nan
    try: return np.corrcoef(a[mask], b[mask])[0, 1]
    except Exception: return np.nan

# ----------------- MAIN -----------------
def main():
    df = load_data(CSV_PATH)
    if df is None:
        return

    # Step 1: Data types / missing / duplicates
    dtypes = pd.DataFrame(df.dtypes, columns=["dtype"]); save_table(dtypes, "00_dtypes")
    missing = df.isna().sum().to_frame("missing_count"); save_table(missing, "01_missing_counts")
    dup_count = int(df.duplicated().sum())
    with open(OUTPUT_DIR / "02_duplicates.txt", "w", encoding="utf-8") as f:
        f.write(f"Duplicate rows: {dup_count}\n")
    print(f"Saved: {OUTPUT_DIR / '02_duplicates.txt'}")

    # Step 2: Summary stats (overall + by speed)
    summary = df.describe(include="all").transpose(); save_table(summary, "03_summary_overall")
    if ENGINE_SPEED_COL in df.columns:
        summaries = []
        for spd, sub in df.groupby(ENGINE_SPEED_COL):
            s = sub.describe().transpose(); s["engine_speed"] = spd; summaries.append(s)
        if summaries:
            save_table(pd.concat(summaries), "04_summary_by_speed")

    # Step 3: Distributions & boxplots
    for var in DIST_VARS:
        if var in df.columns:
            plot_histogram(df[var], f"Distribution of {var}", f"10_hist_{var}.png")
            if ENGINE_SPEED_COL in df.columns:
                plot_box_by_speed(df, var, ENGINE_SPEED_COL, f"{var} by {ENGINE_SPEED_COL}", f"11_box_{var}_by_{ENGINE_SPEED_COL}.png")

    # Step 4: Correlations (overall + per speed) — Pearson, Spearman, plus p-values
    corr_cols = [c for c in INPUT_COLS + OUTPUT_COLS if c in df.columns]
    if corr_cols:
        r_all, p_all = corr_with_pvalues(df[corr_cols])
        s_all = df[corr_cols].corr(method="spearman")
        save_table(r_all, "20_corr_pearson_overall")
        save_table(s_all, "21_corr_spearman_overall")
        save_table(p_all, "21_corr_pvalues_overall")
        plot_corr_heatmap(r_all, "Pearson Correlation (Overall)", "22_heatmap_pearson_overall.png")

        # per-speed
        if ENGINE_SPEED_COL in df.columns:
            for spd, sub in df.groupby(ENGINE_SPEED_COL):
                r_spd, p_spd = corr_with_pvalues(sub[corr_cols])
                s_spd = sub[corr_cols].corr(method="spearman")
                save_table(r_spd, f"20_corr_pearson_speed_{spd}")
                save_table(s_spd, f"21_corr_spearman_speed_{spd}")
                save_table(p_spd, f"21_corr_pvalues_speed_{spd}")
                plot_corr_heatmap(r_spd, f"Pearson Correlation (Speed {spd})", f"22_heatmap_pearson_speed_{spd}.png")

    # Step 5: Feature ranking (overall + per speed)
    rank_overall = rank_inputs_by_corr(df, INPUT_COLS, OUTPUT_COLS); save_table(rank_overall, "30_feature_ranking_overall")
    if ENGINE_SPEED_COL in df.columns:
        ranks = []
        for spd, sub in df.groupby(ENGINE_SPEED_COL):
            r = rank_inputs_by_corr(sub, INPUT_COLS, OUTPUT_COLS); r["engine_speed"] = spd; ranks.append(r)
        if ranks: save_table(pd.concat(ranks, ignore_index=True), "31_feature_ranking_by_speed")

    # Step 6: Top-K overall scatter plots (coloured by speed if available)
    for y in OUTPUT_COLS:
        subrank = rank_overall[rank_overall["output"] == y].head(TOPK)
        for _, row in subrank.iterrows():
            x = row["input"]
            if x in df.columns and y in df.columns:
                scatter_xy(df, x, y, ENGINE_SPEED_COL if ENGINE_SPEED_COL in df.columns else None,
                           f"{y} vs {x} (Overall)", f"40_scatter_{y}_vs_{x}_overall.png")

    # Step 7: Per-speed scatter plots (Top-K per output) + strong output↔output
    if ENGINE_SPEED_COL in df.columns:
        for spd, sub in df.groupby(ENGINE_SPEED_COL):
            rsub = rank_inputs_by_corr(sub, INPUT_COLS, OUTPUT_COLS)
            for y in OUTPUT_COLS:
                top_inputs = rsub[rsub["output"] == y].head(TOPK)["input"].tolist()
                for x in top_inputs:
                    if x in sub.columns and y in sub.columns:
                        scatter_xy(sub, x, y, None,
                                   f"{y} vs {x} (Speed {spd})", f"41_scatter_{y}_vs_{x}_speed_{spd}.png")

        # 7b: strong input→output per speed
        save_per_speed_scatterplots(df=df, inputs=INPUT_COLS, outputs=OUTPUT_COLS,
                                    speed_col=ENGINE_SPEED_COL, outdir=OUTPUT_DIR, rmin=0.60, topk=5)

        # 7c: strong output↔output per speed
        OO_THRESH = 0.75
        for spd, sub in df.groupby(ENGINE_SPEED_COL):
            sub_num = sub.select_dtypes(include=[np.number])
            outs_here = [c for c in OUTPUT_COLS if c in sub_num.columns]
            strong_pairs = []
            for i in range(len(outs_here)):
                for j in range(i+1, len(outs_here)):
                    y, x = outs_here[i], outs_here[j]
                    r = pearson_r_series(sub_num[x], sub_num[y])
                    if np.isfinite(r) and abs(r) >= OO_THRESH:
                        strong_pairs.append((x, y, r))
            for x, y, r in sorted(strong_pairs, key=lambda t: abs(t[2]), reverse=True):
                scatter_xy(sub, x=x, y=y, color_by=None,
                           title=f"{y} vs {x} (Speed {spd})  r={r:.2f}",
                           fname=f"52_scatter_speed_{spd}_{y}_vs_{x}.png", fit=True)

    print("\n✔️ EDA Steps 1–7 complete. Outputs saved to:", OUTPUT_DIR)

main()


Columns: ['timestamp', 'run_id', 'rpm', 'torque_Nm', 'imep_bar', 'fuel_flow_kgph', 'air_flow_kgph', 'lambda', 'egr_frac', 'map_bar_abs', 'mat_C', 'coolant_in_C', 'coolant_out_C', 'exhaust_T_C', 'oil_T_C', 'prr_bar_per_deg', 'CO2_dry_pct', 'NOx_ppm', 'CO_ppm', 'HC_ppm', 'knock_index']
            timestamp  run_id   rpm   torque_Nm  imep_bar  fuel_flow_kgph  \
0 2025-01-01 12:00:00       1  1497  765.745845  7.815380       43.837174   
1 2025-01-01 12:00:00       1  1525  778.313244  7.795354       37.422822   
2 2025-01-01 12:00:00       1  1507  773.855949  7.726995       38.494482   
3 2025-01-01 12:00:00       1  1484  792.421523  7.906442       37.948083   
4 2025-01-01 12:00:00       1  1500  764.448437  7.653660       30.062155   

   air_flow_kgph    lambda  egr_frac  map_bar_abs  ...  coolant_in_C  \
0    1273.954349  1.439524  0.145030     2.385132  ...     85.241962   
1    1077.758265  1.459768  0.152575     2.447865  ...     83.978448   
2    1041.004571  1.376404  0.145058

In [2]:
!pip install xlsxwriter

In [4]:
# takes all cleaned outputs from above and sorts into one xls file
# Overwrite: re-running this cell overwrites engine_analysis_export.xlsx. Close it in Excel before re-running.
import pandas as pd
from pathlib import Path
import re
from xlsxwriter.utility import xl_rowcol_to_cell

# Use the same OUTPUT_DIR and CSV_PATH defined earlier (Cell 1)
# If you re-open this notebook fresh, re-run Cell 1 first.

OUTPUT_DIR = Path(r"C:\Users\HP\OneDrive\GitHub\Engine_sensor_analytics\out")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EXPORT_XLSX = OUTPUT_DIR / "engine_analysis_export.xlsx"
MIN_BIN_COUNT = 4   # speeds with fewer rows are skipped

# ---------- formatting helpers ----------
def _body_range(df):
    n_rows, n_cols = df.shape
    return 1, 1, n_rows, n_cols

def apply_corr_format(writer, sheet_name, df):
    ws = writer.sheets[sheet_name]; wb = writer.book
    fr, fc, lr, lc = _body_range(df)
    tl = xl_rowcol_to_cell(fr, fc, row_abs=False, col_abs=False)
    rng = (fr, fc, lr, lc)
    fmt_red   = wb.add_format({'bg_color': '#ea9999'})
    fmt_amber = wb.add_format({'bg_color': '#f6b26b'})
    fmt_green = wb.add_format({'bg_color': '#93c47d'})
    fmt_blue  = wb.add_format({'bg_color': '#6fa8dc'})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=ABS({tl})>=0.90","format": fmt_blue,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=AND(ABS({tl})>=0.50,ABS({tl})<0.90)","format": fmt_green,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=AND(ABS({tl})>=0.30,ABS({tl})<0.50)","format": fmt_amber,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=AND(NOT(ISBLANK({tl})),ABS({tl})<0.30)","format": fmt_red,"stop_if_true": True})

def apply_pval_format(writer, sheet_name, df):
    ws = writer.sheets[sheet_name]; wb = writer.book
    fr, fc, lr, lc = _body_range(df)
    tl = xl_rowcol_to_cell(fr, fc, row_abs=False, col_abs=False)
    rng = (fr, fc, lr, lc)
    fmt_ns   = wb.add_format({'bg_color': '#ea9999'})
    fmt_sig  = wb.add_format({'bg_color': '#f6b26b'})
    fmt_str  = wb.add_format({'bg_color': '#93c47d'})
    fmt_vstr = wb.add_format({'bg_color': '#6fa8dc'})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"={tl}>=0.05","format": fmt_ns,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=AND({tl}<0.05,{tl}>=0.01)","format": fmt_sig,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"=AND({tl}<0.01,{tl}>=0.001)","format": fmt_str,"stop_if_true": True})
    ws.conditional_format(*rng, {"type": "formula","criteria": f"={tl}<0.001","format": fmt_vstr,"stop_if_true": True})

def clean_sheet_name(name: str) -> str:
    name = re.sub(r'[\[\]:*?/\\]', '_', name)
    return name[:31]

def pretty_from_stem(stem: str) -> str:
    return clean_sheet_name(stem.replace('_', ' '))

# --- Figure out which speed bins exist & how many rows in each ---
try:
    df_raw = pd.read_csv(CSV_PATH)  # CSV_PATH defined in Cell 1
    bin_col_candidates = [c for c in df_raw.columns if "rpm_binned" in c]
    BIN_COL = bin_col_candidates[0] if bin_col_candidates else "rpm_binned"
    if BIN_COL not in df_raw.columns and "rpm" in df_raw.columns:
        df_raw["rpm_binned"] = df_raw["rpm"].round(-2).astype("Int64")
        BIN_COL = "rpm_binned"
    bin_counts = df_raw[BIN_COL].value_counts().sort_index()
except Exception:
    bin_counts = pd.Series(dtype=int)

def bin_ok_from_stem(stem: str) -> bool:
    m = re.search(r"_speed_(\d+)", stem)
    if not m or bin_counts.empty:
        return True
    spd = int(m.group(1))
    return bin_counts.get(spd, 0) >= MIN_BIN_COUNT

# ---- Priority groups (dynamic: overall first, then per-speed in numeric order) ----
# add overall sheets explicitly; per-speed will be picked up automatically from filenames.
priority_blocks = [
    ["00_dtypes", "01_missing_counts", "02_duplicates",
     "03_summary_overall", "04_summary_by_speed"],
    ["20_corr_pearson_overall", "21_corr_spearman_overall", "21_corr_pvalues_overall"],
    ["30_feature_ranking_overall", "31_feature_ranking_by_speed"],
]

# Gather files
csv_files = sorted(OUTPUT_DIR.glob("*.csv"), key=lambda p: p.stem)
png_files = sorted(OUTPUT_DIR.glob("*.png"), key=lambda p: p.stem)
csv_map = {p.stem: p for p in csv_files}

# Group images (per-speed & others)
speed_imgs = {}
other_imgs = []
for p in png_files:
    m = re.match(r"5[12]_scatter_speed_(\d+)_", p.stem)
    if m:
        spd = int(m.group(1))
        speed_imgs.setdefault(spd, []).append(p)
    else:
        other_imgs.append(p)

with pd.ExcelWriter(EXPORT_XLSX, engine="xlsxwriter") as writer:
    written = set()

    # 1) Priority blocks first
    for block in priority_blocks:
        for stem in block:
            if stem in csv_map and bin_ok_from_stem(stem):
                df = pd.read_csv(csv_map[stem])
                sheet_name = pretty_from_stem(stem)
                df.to_excel(writer, index=False, sheet_name=sheet_name)
                written.add(stem)
                low = stem.lower()
                if ("corr" in low) and ("pvalues" not in low):
                    apply_corr_format(writer, sheet_name, df)
                elif "pvalues" in low:
                    apply_pval_format(writer, sheet_name, df)

    # 2) Then all remaining CSVs (includes per-speed automatically)
    for csv_path in csv_files:
        stem = csv_path.stem
        if stem in written or not bin_ok_from_stem(stem):
            continue
        df = pd.read_csv(csv_path)
        sheet_name = pretty_from_stem(stem)
        base, i = sheet_name, 1
        while sheet_name in writer.sheets:
            sheet_name = clean_sheet_name(f"{base}_{i}"); i += 1
        df.to_excel(writer, index=False, sheet_name=sheet_name)
        low = stem.lower()
        if ("corr" in low) and ("pvalues" not in low):
            apply_corr_format(writer, sheet_name, df)
        elif "pvalues" in low:
            apply_pval_format(writer, sheet_name, df)

    # 3) Per-speed plot sheets
    if speed_imgs:
        workbook = writer.book
        col_spacing = 8; row_spacing = 20; scale = 0.85
        for spd in sorted(speed_imgs):
            imgs = sorted(speed_imgs[spd], key=lambda p: p.stem)
            sheet_title = clean_sheet_name(f"PerSpeed_{spd}")
            ws = workbook.add_worksheet(sheet_title)
            writer.sheets[sheet_title] = ws
            img_idx = 0
            for img in imgs:
                row_block = img_idx // 2
                col_block = img_idx % 2
                row = row_block * row_spacing
                col = col_block * col_spacing
                ws.insert_image(row, col, str(img), {'x_scale': scale, 'y_scale': scale})
                img_idx += 1

    # 4) Everything else in paged Plots_1, Plots_2, ...
    if other_imgs:
        workbook = writer.book
        max_per_sheet = 18
        col_spacing = 8
        row_spacing  = 20
        scale = 0.85
        sheet_idx = 1
        img_idx_on_sheet = 0
        ws = None
        for img in sorted(other_imgs, key=lambda p: p.stem):
            if img_idx_on_sheet == 0:
                sheet_title = clean_sheet_name(f"Plots_{sheet_idx}")
                ws = workbook.add_worksheet(sheet_title)
                writer.sheets[sheet_title] = ws
                sheet_idx += 1
            row_block = img_idx_on_sheet // 2
            col_block = img_idx_on_sheet % 2
            row = row_block * row_spacing
            col = col_block * col_spacing
            ws.insert_image(row, col, str(img), {'x_scale': scale, 'y_scale': scale})
            img_idx_on_sheet += 1
            if img_idx_on_sheet >= max_per_sheet:
                img_idx_on_sheet = 0

print(f"✅ Exported {len(list(OUTPUT_DIR.glob('*.csv')))} tables and {len(list(OUTPUT_DIR.glob('*.png')))} plots to:\n{EXPORT_XLSX}")


✅ Exported 18 tables and 214 plots to:
C:\Users\HP\OneDrive\GitHub\Engine_sensor_analytics\out\engine_analysis_export.xlsx
